## Data Preprocessing

In [2]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("Crime_Data_from_2020_to_Present.csv")

# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^a-z0-9_]', '', regex=True)

# Parse and clean 'date_rptd' and 'date_occ' columns
for col in ['date_rptd', 'date_occ']:
    df[col] = pd.to_datetime(df[col], errors='coerce')  # Convert to datetime
    df[col] = df[col].dt.date  

# Convert 'time_occ' column into standard time format 'HH:MM' string
def parse_time(t):
    try:
        t = int(t)
        if t == 2400:  
            t = 0
        hour = t // 100
        minute = t % 100
        if hour > 23 or minute > 59:
            return np.nan
        return f"{hour:02d}:{minute:02d}"  
    except:
        return np.nan

df['time_occ_clean'] = df['time_occ'].apply(parse_time)


# Extract year, month, day, and hour and create new columns
df['year'] = pd.to_datetime(df['date_occ'], errors='coerce').dt.year
df['month'] = pd.to_datetime(df['date_occ'], errors='coerce').dt.month
df['day'] = pd.to_datetime(df['date_occ'], errors='coerce').dt.day
df['hour'] = df['time_occ_clean'].str.split(':').str[0].astype(float)

# Drop unnecessary columns 
drop_cols= ['crm_cd_2', 'crm_cd_3', 'crm_cd_4', 'status', 'mocodes', 'weapon_used_cd', 'part_12']
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

# Handle missing values
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = df.select_dtypes(include=[np.number]).columns

df[cat_cols] = df[cat_cols].fillna('UNKNOWN')
for col in num_cols:
    if df[col].isna().sum() > 0:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)


# Handle invalid entries of 'vict_age' values
if 'vict_age' in df.columns:
    invalid_age = (df['vict_age'] < 0) | (df['vict_age'] > 120)
    median_age = df.loc[~invalid_age, 'vict_age'].median()
    df.loc[invalid_age, 'vict_age'] = median_age
    print(f"Replaced invalid victim ages (<0 or >120) with median = {median_age}")

df = df.reset_index(drop=True)

# Saving the preprocessed dataset for future use
df.to_csv("Crime_Data_Preprocessed.csv", index=False)
print("Preprocessed dataset saved as 'Crime_Data_Preprocessed.csv'")
print("\n Dataset shape:", df.shape)

df.head()


C:\Users\sarik\AppData\Local\Temp\ipykernel_13672\1492066144.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')  # Convert to datetime
C:\Users\sarik\AppData\Local\Temp\ipykernel_13672\1492066144.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')  # Convert to datetime


Replaced invalid victim ages (<0 or >120) with median = 30.0
Preprocessed dataset saved as 'Crime_Data_Preprocessed.csv'

 Dataset shape: (1004991, 26)


,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,crm_cd,crm_cd_desc,vict_age,...,crm_cd_1,location,cross_street,lat,lon,time_occ_clean,year,month,day,hour
0,211507896,2021-04-11,2020-11-07,845,15,N Hollywood,1502,354,THEFT OF IDENTITY,31,...,354.0,7800 BEEMAN AV,UNKNOWN,34.2124,-118.4092,08:45,2020,11,7,8.0
1,201516622,2020-10-21,2020-10-18,1845,15,N Hollywood,1521,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",32,...,230.0,ATOLL AV,N GAULT,34.1993,-118.4203,18:45,2020,10,18,18.0
2,240913563,2024-12-10,2020-10-30,1240,9,Van Nuys,933,354,THEFT OF IDENTITY,30,...,354.0,14600 SYLVAN ST,UNKNOWN,34.1847,-118.4509,12:40,2020,10,30,12.0
3,210704711,2020-12-24,2020-12-24,1310,7,Wilshire,782,331,THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND ...,47,...,331.0,6000 COMEY AV,UNKNOWN,34.0339,-118.3747,13:10,2020,12,24,13.0
4,201418201,2020-10-03,2020-09-29,1830,14,Pacific,1454,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),63,...,420.0,4700 LA VILLA MARINA,UNKNOWN,33.9813,-118.4350,18:30,2020,9,29,18.0
